In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy.stats import norm, gamma
from scipy.stats._distn_infrastructure import rv_frozen
import matplotlib
from tqdm import tqdm
import mercury as mr
from IPython.display import HTML
from ipywidgets import widgets

from RWMetropolis import Target, UpdateDist

In [2]:
_ = mr.Note(text="Gamma prior parameters")
alpha = mr.Numeric(value=1, min=0.1, max=10, label="α", step=0.1)
beta = mr.Numeric(value=1, min=0.1, max=10, label="β", step=0.1)
_ = mr.Note(text="Gamma likelihood shape")
v = mr.Numeric(value=1, min=0.1, max=10, label="v", step=0.1)
x = [1]
n = len(x)
_ = mr.Note(text="Algorithm options")
seed = mr.Numeric(value=1, min=0, max=2 ** 30, label="Random seed")
x_0 = mr.Numeric(value=1, min=0.1, max=5, label="x_0", step=0.1)
samples = mr.Slider(value=100, min=1, max=1000, label="samples", step=1)
proposal_sigma = mr.Numeric(
    value=0.4, min=0.1, max=2, label="Proposal standard deviation", step=0.1
)
_ = mr.Note(text="Plot options")
bins = mr.Slider(value=35, min=5, max=60, label="Histogram bins", step=1)

Gamma prior parameters

mercury.Numeric

mercury.Numeric

Gamma likelihood shape

mercury.Numeric

...

mercury.Numeric

mercury.Slider

mercury.Numeric

Plot options

mercury.Slider

In [3]:
mr.Markdown(
    text="""See [this page](https://areding.github.io/6420-pymc/unit5/Unit5-Metropolis.html) for the algorithm description.

Let our model be a Gamma-Gamma conjugate model, where:

$$
\\begin{align*}
X_i | \\beta &\\sim \\text{Ga}(v, \\theta) \\\\
\\theta &\\sim \\text{Ga}(\\alpha, \\beta)
\\end{align*}
$$

We'll just have a single datapoint, $x=1$, for simplicity.

For our target $\\pi(x)$, we have:

$$\pi(\\theta|x) \\propto \\theta^{vn + \\alpha - 1} e^{-\\theta(\\beta + \\sum_{i=1}^{n}X_i)}$$

Our true posterior by conjugacy is:

$$Ga(\\alpha + nv, \\beta + \\sum_{i=1}^{n}X_i)$$

We'll use a $N(0, \\sigma^2)$ proposal.
"""
)

See [this page](https://areding.github.io/6420-pymc/unit5/Unit5-Metropolis.html) for the algorithm description.

Let our model be a Gamma-Gamma conjugate model, where:

$$
\begin{align*}
X_i | \beta &\sim \text{Ga}(v, \theta) \\
\theta &\sim \text{Ga}(\alpha, \beta)
\end{align*}
$$

We'll just have a single datapoint, $x=1$, for simplicity.

For our target $\pi(x)$, we have:

$$\pi(\theta|x) \propto \theta^{vn + \alpha - 1} e^{-\theta(\beta + \sum_{i=1}^{n}X_i)}$$

Our true posterior by conjugacy is:

$$Ga(\alpha + nv, \beta + \sum_{i=1}^{n}X_i)$$

We'll use a $N(0, \sigma^2)$ proposal.


In [5]:
true_posterior = gamma(a=alpha.value + n * v.value, scale=1.0 / (beta.value + sum(x)))
target = Target(v.value, alpha.value, beta.value, data=x)

threshold = 0.999
upper_bound = round(true_posterior.ppf(threshold))

fig, ax = plt.subplots()

ud = UpdateDist(
    ax,
    x_0.value,
    target,
    true_posterior,
    proposal_sigma=proposal_sigma.value,
    lower_bound=0.0,
    upper_bound=upper_bound,
    bins=bins.value,
    sample_count=samples.value,
    seed=int(seed.value),
)
plt.close(fig)

matplotlib.rcParams["animation.embed_limit"] = 500
anim = FuncAnimation(fig, ud, frames=samples.value, interval=100, blit=True)
widgets.HTML(value=anim.to_jshtml())
#HTML(anim.to_jshtml())

/Users/aaron/mambaforge/envs/mercury/lib/python3.10/site-packages/numpy/lib/histograms.py:885: RuntimeWarning: divide by zero encountered in divide
  return n/db/n.sum(), bin_edges
/Users/aaron/mambaforge/envs/mercury/lib/python3.10/site-packages/numpy/lib/histograms.py:885: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges


HTML(value='\n<link rel="stylesheet"\nhref="https://maxcdn.bootstrapcdn.com/font-awesome/4.4.0/css/font-awesom…